# Overview
Try to analyze and manipulate the data from _imports-85.data_ (CSV) and *train_data.json* to improve my data-munging skills in __*python*__ and __*Spark*__.

## Goal 1 - Find out how the risk breaks down by engine

Engine type is the fifth column and has values; `std` and `turbo`

In [1]:
rdd = spark.sparkContext.textFile('../data/imports-85.data').map(lambda line: line.split(','))
print rdd.count()
print rdd.take(2)

205
[[u'3', u'?', u'alfa-romero', u'gas', u'std', u'two', u'convertible', u'rwd', u'front', u'88.60', u'168.80', u'64.10', u'48.80', u'2548', u'dohc', u'four', u'130', u'mpfi', u'3.47', u'2.68', u'9.00', u'111', u'5000', u'21', u'27', u'13495'], [u'3', u'?', u'alfa-romero', u'gas', u'std', u'two', u'convertible', u'rwd', u'front', u'88.60', u'168.80', u'64.10', u'48.80', u'2548', u'dohc', u'four', u'130', u'mpfi', u'3.47', u'2.68', u'9.00', u'111', u'5000', u'21', u'27', u'16500']]


In [2]:
groupData = rdd.map(lambda line: ((line[0], line[4]), 1))
groupData.take(2)

[((u'3', u'std'), 1), ((u'3', u'std'), 1)]

In [3]:
groupByRiskAndEngine = groupData.reduceByKey(lambda x, y: x+y)
groupByRiskAndEngine.cache()
groupByRiskAndEngine.take(2)

[((u'3', u'std'), 19), ((u'0', u'turbo'), 11)]

In [4]:
print groupByRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()

[((u'0', u'std'), 56), ((u'1', u'std'), 47), ((u'2', u'std'), 30), ((u'3', u'std'), 19), ((u'-1', u'std'), 14), ((u'0', u'turbo'), 11), ((u'-1', u'turbo'), 8), ((u'3', u'turbo'), 8), ((u'1', u'turbo'), 7), ((u'2', u'turbo'), 2), ((u'-2', u'std'), 2), ((u'-2', u'turbo'), 1)]


In [5]:
stdRiskAndEngine = groupByRiskAndEngine.filter(lambda pair: pair[0][1]=='std')
turboRiskAndEngine = groupByRiskAndEngine.filter(lambda pair: pair[0][1]=='turbo')

stdRiskAndEngine.cache()
turboRiskAndEngine.cache()

print('Sorted Risk for Standard Engines: \n%s\n' % (stdRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()))

print('Sorted Risk for Turbo Engines: \n%s' % (turboRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()))


Sorted Risk for Standard Engines: 
[((u'0', u'std'), 56), ((u'1', u'std'), 47), ((u'2', u'std'), 30), ((u'3', u'std'), 19), ((u'-1', u'std'), 14), ((u'-2', u'std'), 2)]

Sorted Risk for Turbo Engines: 
[((u'0', u'turbo'), 11), ((u'-1', u'turbo'), 8), ((u'3', u'turbo'), 8), ((u'1', u'turbo'), 7), ((u'2', u'turbo'), 2), ((u'-2', u'turbo'), 1)]


In [6]:
stdRisk = stdRiskAndEngine.map(lambda pair: int(pair[0][0]) * pair[1]).reduce(lambda x,y:x+y)
stdCount = stdRiskAndEngine.map(lambda pair: int(pair[1])).reduce(lambda x,y:x+y)
stdRiskWeighted = float(stdRisk)/float(stdCount)

turboRisk = turboRiskAndEngine.map(lambda pair: int(pair[0][0]) * pair[1]).reduce(lambda x,y:x+y)
turboCount = turboRiskAndEngine.map(lambda pair: int(pair[1])).reduce(lambda x,y:x+y)
turboRiskWeighted = float(turboRisk)/float(turboCount)

print('The std cummulative risk is: %d and the turbo cummulative risk is: %d' % (stdRisk, turboRisk))
print('The std weighted risk is (%d): %f and the turbo weighted risk is(%d): %f' % (stdCount, stdRiskWeighted, turboCount, turboRiskWeighted))

The std cummulative risk is: 146 and the turbo cummulative risk is: 25
The std weighted risk is (168): 0.869048 and the turbo weighted risk is(37): 0.675676


## Goal 2 - Evaluate Cross Val Model

In [7]:
import cPickle
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from score_auto_gbm.FeatureTransformer import FeatureTransformer

In [14]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [8]:
with open("./score_auto_gbm/gbmFit.pkl", "rb") as pickle_file:
    gbmFit = cPickle.load(pickle_file)

In [38]:
data = pd.read_csv('../data/imports-85.data', header=None)
data.columns = [u'risk', u'normalizedLosses', u'make', u'fuelType', u'aspiration', u'numDoors',
       u'bodyStyle', u'driveWheels', u'engineLocation', u'wheelBase',
       u'length', u'width', u'height', u'curbWeight', u'engineType',
       u'numCylinders', u'engineSize', u'fuelSystem', u'bore', u'stroke',
       u'compressionRatio', u'horsepower', u'peakRPM', u'cityMPG', u'highwayMPG', u'price']
print data.index.size
data.head()

205


,risk,normalizedLosses,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [21]:
trainData = data.ix[:,2:]

trainData.head()

,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,length,width,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [70]:
predictedActualTuple = []
for idx, row in enumerate(trainData.to_dict(orient='records')):
    try:
        prediction = list(gbmFit.predict(pd.DataFrame([row])))[0]
    except ValueError:
        print ('There was an issue predicting row: %d' % (idx + 1))
        continue
    
    if is_number(data.ix[idx,0]) == True and is_number(prediction) == True:
       predictedActualTuple.extend([(float(prediction), float(data.ix[idx,0]))])

numberPredictions = len(predictedActualTuple)
squaredError = map(lambda errorTuple: (errorTuple[0] - errorTuple[1])**2, predictedActualTuple)
rmse = (sum(squaredError)/numberPredictions)**0.5
maxPred, maxAct = reduce(lambda x, y:  x if abs(x[0]-x[1]) > abs(y[0]-y[1]) else y, predictedActualTuple)

print ('\nThe number of instances was: %d.\nThe squaured error was: %f.\nRMSE: %f.\nMaxed squared error: %f.\nPredicted & Acutal for max diff: %f & %f' 
       % (number,squareError,rmse, max(squaredError), maxPred, maxAct))


There was an issue predicting row: 10
There was an issue predicting row: 45
There was an issue predicting row: 46
There was an issue predicting row: 56
There was an issue predicting row: 57
There was an issue predicting row: 58
There was an issue predicting row: 59
There was an issue predicting row: 130
There was an issue predicting row: 131
There was an issue predicting row: 132

The number of instances was: 205.
The squaured error was: 10.487928.
RMSE: 0.231914.
Maxed squared error: 3.999332.
Predicted & Acutal for max diff: 0.999833 & -1.000000


In [61]:
test = []
test.extend([(0.99,1.00), (1.01,0.98)])
testSquareError = map(lambda x: (x[1] - x[0])**2,test)
print test
print testSquareError
print sum(testSquareError)
print round(float(reduce(lambda x,y: float(x+y), testSquareError)), 10)

[(0.99, 1.0), (1.01, 0.98)]
[0.00010000000000000018, 0.0009000000000000016]
0.001
0.001


In [67]:
# Print troubled rows. They are the ones with ? in them.
# Then why does
trainData.ix[[9,44,45,55,56,58,129,130,131],:]

,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,length,width,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
9,audi,gas,turbo,two,hatchback,4wd,front,99.5,178.2,67.9,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?
44,isuzu,gas,std,two,sedan,fwd,front,94.5,155.9,63.6,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
45,isuzu,gas,std,four,sedan,fwd,front,94.5,155.9,63.6,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
55,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,70,4bbl,?,?,9.4,101,6000,17,23,10945
56,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,70,4bbl,?,?,9.4,101,6000,17,23,11845
58,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,80,mpfi,?,?,9.4,135,6000,16,23,15645
129,porsche,gas,std,two,hatchback,rwd,front,98.4,175.7,72.3,...,203,mpfi,3.94,3.11,10.0,288,5750,17,28,?
130,renault,gas,std,four,wagon,fwd,front,96.1,181.5,66.5,...,132,mpfi,3.46,3.90,8.7,?,?,23,31,9295
131,renault,gas,std,two,hatchback,fwd,front,96.1,176.8,66.6,...,132,mpfi,3.46,3.90,8.7,?,?,23,31,9895


In [69]:
for i in range(10):
    if i % 2 == 0:
        print('This is even.', i)
        continue
    
    print('This is not even', i)

('This is even.', 0)
('This is not even', 1)
('This is even.', 2)
('This is not even', 3)
('This is even.', 4)
('This is not even', 5)
('This is even.', 6)
('This is not even', 7)
('This is even.', 8)
('This is not even', 9)


## Goal 3 - Use munging & math to evaluate how good RMSE was